In [1]:
import pandas as pd

In [2]:
# Import TOPO1K and OSM matching file as dataframe and edit column names

df_ibb_osm = pd.read_csv("data/match_ibb_osm.csv")[['fid_1','fid_2']]
df_ibb_osm.rename(columns={'fid_1':'reference','fid_2':'target'},inplace=True)

In [3]:
# Calculate relation counts between features

df_ibb_osm['count_ref'] = df_ibb_osm['reference'].map(df_ibb_osm['reference'].value_counts())
df_ibb_osm['count_tar'] = df_ibb_osm['target'].map(df_ibb_osm['target'].value_counts())

In [4]:
# Identify relation types

a=[]
for i,row in df_ibb_osm.iterrows():
    if row['count_ref'] == 1 and row['count_tar'] == 1:
        a.append('sameAsRelation')
    elif row['count_ref'] == 1 and row['count_tar'] > 1:
        a.append('subSetOf')
    elif row['count_ref'] > 1 and row['count_tar'] == 1:
        a.append('superSetOf')
    elif row['count_ref'] > 1 and row['count_tar'] > 1:
        a.append('multipleRelation')
        
df_ibb_osm['type'] = a

In [6]:
# If any building has a multiple relation with other buildings, all the other relations must also be multiple with them

mrr = list(set(df_ibb_osm[df_ibb_osm['type'] == 'multipleRelation']['reference'].to_list()))
mrt = list(set(df_ibb_osm[df_ibb_osm['type'] == 'multipleRelation']['target'].to_list()))
not_mrr = df_ibb_osm[df_ibb_osm['type'] != 'multipleRelation']['reference']
not_mrt = df_ibb_osm[df_ibb_osm['type'] != 'multipleRelation']['target']
change_index = list(set(not_mrr[not_mrr.isin(mrr)].index.to_list() + not_mrt[not_mrt.isin(mrt)].index.to_list()))
df_ibb_osm.loc[change_index,'type'] = 'multipleRelation'

In [8]:
# Import OSM and TOPO25K building matching file as dataframe and edit column names

df_osm_hgm = pd.read_csv("data/match_osm_hgm.csv")[['fid_1','fid_2']]
df_osm_hgm.rename(columns={'fid_1':'reference','fid_2':'target'},inplace=True)

In [9]:
# Calculate relation counts between features

df_osm_hgm['count_ref'] = df_osm_hgm['reference'].map(df_osm_hgm['reference'].value_counts())
df_osm_hgm['count_tar'] = df_osm_hgm['target'].map(df_osm_hgm['target'].value_counts())

In [10]:
# Identify relation types

a=[]
for i,row in df_osm_hgm.iterrows():
    if row['count_ref'] == 1 and row['count_tar'] == 1:
        a.append('sameAsRelation')
    elif row['count_ref'] == 1 and row['count_tar'] > 1:
        a.append('subSetOf')
    elif row['count_ref'] > 1 and row['count_tar'] == 1:
        a.append('superSetOf')
    elif row['count_ref'] > 1 and row['count_tar'] > 1:
        a.append('multipleRelation')
        
df_osm_hgm['type'] = a

In [12]:
# If any building has a multiple relation with other buildings, all the other relations must also be multiple with them

mrr = list(set(df_osm_hgm[df_osm_hgm['type'] == 'multipleRelation']['reference'].to_list()))
mrt = list(set(df_osm_hgm[df_osm_hgm['type'] == 'multipleRelation']['target'].to_list()))
not_mrr = df_osm_hgm[df_osm_hgm['type'] != 'multipleRelation']['reference']
not_mrt = df_osm_hgm[df_osm_hgm['type'] != 'multipleRelation']['target']
change_index = list(set(not_mrr[not_mrr.isin(mrr)].index.to_list() + not_mrt[not_mrt.isin(mrt)].index.to_list()))
df_osm_hgm.loc[change_index,'type'] = 'multipleRelation'

In [14]:
# Import OSM and TOPO25K settlement area matching file as dataframe and edit column names

df_osm_hgm_yer = pd.read_csv("data/match_osm_hgm_yer.csv")[['fid_1','fid_2']]
df_osm_hgm_yer.rename(columns={'fid_1':'reference','fid_2':'target'},inplace=True)

In [15]:
# Calculate relation counts between features

df_osm_hgm_yer['count_ref'] = df_osm_hgm_yer['reference'].map(df_osm_hgm_yer['reference'].value_counts())
df_osm_hgm_yer['count_tar'] = df_osm_hgm_yer['target'].map(df_osm_hgm_yer['target'].value_counts())

In [16]:
# Identify relation types

a=[]
for i,row in df_osm_hgm_yer.iterrows():
    if row['count_ref'] == 1 and row['count_tar'] > 1:
        a.append('isPartOf')
    elif row['count_ref'] > 1 and row['count_tar'] == 1:
        a.append('hasPart')
        
df_osm_hgm_yer['type'] = a

In [19]:
# create resulting dataframes

rel= pd.concat([df_ibb_osm, df_osm_hgm]).reset_index(drop=True)

rel2 = df_osm_hgm_yer

In [21]:
# Create relations types as RDF

relations = ['idt:' + rel['reference'][i] + ' spo:' + rel['type'][i] + 
                 ' idt:' + rel['target'][i] + ' .' for i in range(len(rel))]

relations2 = ['idt:' + rel2['reference'][i] + ' dul:' + rel2['type'][i] + 
                 ' idt:' + rel2['target'][i] + ' .' for i in range(len(rel2))]

In [22]:
# Write RDFs to a file

with open('data/relations.ttl', 'w') as output:
        for i in range(len(relations)):
            output.write('\n' + relations[i] + '\n')
            
        for i in range(len(relations2)):
            output.write('\n' + relations2[i] + '\n')